In [58]:
import pandas as pd
import psycopg2
import time

from datetime import datetime, date
from psycopg2.errors import Error

from sqlalchemy import create_engine  
from sqlalchemy import Column, String, DateTime, Date, BigInteger, Text, Integer, Boolean
from sqlalchemy.ext.declarative import declarative_base  
from sqlalchemy.orm import sessionmaker
from sqlalchemy.sql import func

### Connection Parameters

In [5]:
hostname= 'localhost'
port=5432
user='postgres'
password='postgres'
database='sentiment-dc_development'

## Psycopg2

In [8]:
def create_server_connection(hostname, port, user, password, database):

    try:
        connection = psycopg2.connect(
            host=hostname,
            port=port,
            user=user,
            password=password,
            database=database
        )
        print("Postgres Database connection successful")
    except Error as err:
        print(f"Error: '{err}'")
        raise err

    return connection

In [9]:
conn = create_server_connection(
    hostname=hostname,
    user=user,
    password=password,
    port=port,
    database=database, 
    )

Postgres Database connection successful


In [10]:
def list_tables(conn, query=None):
    
    if query is None:
        query = '''
                SELECT *
                FROM pg_catalog.pg_tables
                WHERE schemaname != 'pg_catalog' AND schemaname != 'information_schema';
                '''

    with conn.cursor() as cursor:
        try: 
            cursor.execute(query)
        except Error as err:
            print(f"Error: '{err}'")
            raise
        
        for table in cursor.fetchall():
            yield table

In [11]:
for table in list_tables(conn):
    print(table)

('public', 'schema_migrations', 'postgres', None, True, False, False, False)
('public', 'ar_internal_metadata', 'postgres', None, True, False, False, False)
('public', 'products', 'postgres', None, True, False, False, False)
('public', 'spaces', 'postgres', None, True, False, True, False)
('public', 'lists', 'postgres', None, True, False, True, False)
('public', 'checklists', 'postgres', None, True, False, True, False)
('public', 'tasks', 'postgres', None, True, False, True, False)
('public', 'users', 'postgres', None, True, False, True, False)
('public', 'items', 'postgres', None, True, False, True, False)
('public', 'tasks_users', 'postgres', None, True, False, False, False)
('public', 'property_settings', 'postgres', None, True, False, False, False)
('public', 'availabilities', 'postgres', None, True, False, False, False)
('public', 'availabilities_users', 'postgres', None, True, False, False, False)
('public', 'branches', 'postgres', None, True, False, True, False)
('public', 'comm

## SQL Alchemy

In [16]:
def create_server_connection(hostname, port, user, password, database):
    
    db_string = f"postgresql://{user}:{password}@{hostname}:{port}/{database}"

    connection = create_engine(db_string)
    print("Postgres Database connection successful")

    return connection

In [17]:
conn = create_server_connection(
    hostname=hostname,
    user=user,
    password=password,
    port=port,
    database=database, 
    )

Postgres Database connection successful


In [18]:
def create_pandas_table(sql_query, database = conn):
    table = pd.read_sql_query(sql_query, database)
    return table

In [19]:
create_pandas_table('''
                SELECT *
                FROM pg_catalog.pg_tables
                WHERE schemaname != 'pg_catalog' AND schemaname != 'information_schema';
                ''', conn )

,schemaname,tablename,tableowner,tablespace,hasindexes,hasrules,hastriggers,rowsecurity
0,public,schema_migrations,postgres,None,True,False,False,False
1,public,ar_internal_metadata,postgres,None,True,False,False,False
2,public,products,postgres,None,True,False,False,False
3,public,spaces,postgres,None,True,False,True,False
4,public,lists,postgres,None,True,False,True,False
5,public,checklists,postgres,None,True,False,True,False
6,public,tasks,postgres,None,True,False,True,False
7,public,users,postgres,None,True,False,True,False
8,public,items,postgres,None,True,False,True,False
9,public,tasks_users,postgres,None,True,False,False,False


In [61]:
base = declarative_base()

In [62]:
class Task(base):
    __tablename__ = 'tasks'
    
    id = Column(BigInteger, primary_key=True, autoincrement=True)
    cid = Column(String)
    name = Column(String)
    description = Column(Text)
    parent = Column(String)
    parent = Column(String)
    status = Column(Integer)
    archived = Column(Boolean)
    due_date = Column(DateTime)
    date_created = Column(DateTime)
    date_closed = Column(DateTime)
    created_at = Column(DateTime, nullable=False, default=func.now())
    updated_at = Column(DateTime, nullable=False, default=func.now())
    list_id = Column(BigInteger)
    priority = Column(Integer)
    branch_id = Column(BigInteger)

In [63]:
Session = sessionmaker(conn)  
session = Session()

base.metadata.create_all(conn)

In [73]:
# Create
def create_obj(obj, session):
    session.add(obj)  
    session.commit()

# Read
def list_obj(table_obj, session):
    objs = session.query(table_obj)
    for obj in objs:
        yield obj

# Update        
def update_obj(obj, session):
    # task.name = "Task Name"  
    session.commit()

# Delete
def delete_obj(obj, session):
    session.delete(obj)
    session.commit()

In [70]:
second_task = Task(
    name='Second Task',
    description='Second task on Sentiment-dc'
)
create_obj(second_task, session)

### Reflect Tables in DB

In [74]:
base.metadata.reflect(conn)

In [76]:
class Users(base):
    __table__ = base.metadata.tables["users"]

In [82]:
class Tasks(base):
        __table__ = base.metadata.tables["tasks"]

In [81]:
for user in list_obj(Users, session):
    print(user.username)

Sue Welch
Ryan Turner
Dominic Hemmings
Abigail Henderson
Karen Bell
Piers Thomson


In [87]:
for task in list_obj(Tasks, session):
    print(task.name)

First Task
Second Task
